## Import modules and network graph

In [1]:
#%% Import modules
import os
import shapely.geometry
import pandas as pd
import uuid
import json
import pathlib

# external
import geojson
import networkx as nx
import shapely.wkt
import geopandas as gpd

# our software
import opentnsim.core as TNcore
import openclsim.model as CLmodel
import openclsim.core as CLcore

import dtv_backend.dtv_backend as backend
import dtv_backend.postprocessing
from dtv_backend.network import network_utilities
from dtv_backend.core import vessels as backendVessels
from dtv_backend.core import sites as backendSites
from dtv_backend.processes.single_run_process_fleet import single_run_process


## Initialize environment with network

In [2]:
#%% Initialize environment
# set up environment
env = backend.provide_environment()

# load the network
backend.load_DTV_network_to_env(env)

Defining simulation environment
Starting (down)loading the network
Network succesfully added to simulation


## Define some locations to use

In [3]:
# some interesting locations
locations = {
    'Transferium Maasvlakte': shapely.geometry.Point(4.087406, 51.936737),
    'Neusse': shapely.geometry.Point(6.708892, 51.215737),
    'Basel': shapely.geometry.Point(7.640572, 47.555449),
    'Nijmegen': shapely.geometry.Point(5.8161152, 51.8570535),
    'Waal at St. Andries': shapely.geometry.Point(5.350339, 51.803965)
}


## Define sites 

In [4]:
#%% define sites

# define origin


origin = backendSites.provideSite(
    env = env, 
    point = locations['Transferium Maasvlakte'], 
    name = 'Transferium Maasvlakte',
    capacity = 10000,
    level = 10000,
    loading_rate = 1000,
    unloading_rate = 1000
)

crane_origin = backendSites.provideSite(
    env = env, 
    point = locations['Transferium Maasvlakte'], 
    name = 'Crane Transferium Maasvlakte',
    capacity = 0,
    level = 0,
    loading_rate = 1000,
    unloading_rate = 1000
)


# define destination
destination = backendSites.provideSite(
    env = env, 
    point = locations['Basel'], 
    name = 'Basel',
    capacity = 10000,
    level = 0,
    loading_rate = 1000,
    unloading_rate = 1000
)

crane_destination = backendSites.provideSite(
    env = env, 
    point = locations['Basel'], 
    name = 'Crane Basel',
    capacity = 0,
    level = 0,
    loading_rate = 1000,
    unloading_rate = 1000
)


## Determine minimal waterlevel based on lobith discharge

In [5]:
lobith_discharge = 1500
max_draught = network_utilities.determine_max_draught_on_path(env.FG, origin, destination, lobith_discharge)
max_draught

3.274149362312395

## Create a vessel

In [6]:
#%% define a vessel

data_110_a = {
    "env": env,
    "name": "NPRC_110_a",
    "geometry": origin.geometry,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 3000,
    "allowable_draught": max_draught,
    "route": None,
    'vessel_type': 'M8',
    'installed_power': 1000,
    'width': 10, 
    'length': 110, 
    'height_empty': 8, 
    'height_full': 4, 
    'draught_empty': 2, 
    'draught_full': 6
}

NPRC_110_a = backendVessels.provideVessel(**data_110_a)


data_110_b = {
    "env": env,
    "name": "NPRC_110_b",
    "geometry": origin.geometry,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 3000,
    "allowable_draught": max_draught,
    "route": None,
    'vessel_type': 'M8',
    'installed_power': 1000,
    'width': 10, 
    'length': 110, 
    'height_empty': 8, 
    'height_full': 4, 
    'draught_empty': 2, 
    'draught_full': 6
}

NPRC_110_b = backendVessels.provideVessel(**data_110_b)
vessels = [NPRC_110_a, NPRC_110_b]

## Functionality to create a single run process but with the moving functionalities of openTNsim

In [7]:

requested_resources = {}
registry = {}
single_run_a, activity_a, while_activity_a  = single_run_process(
    name="single_run_a",
    registry=registry,
    env=env,
    origin=origin,
    destination=destination,
    mover=NPRC_110_a,
    loader=crane_origin,
    unloader=crane_destination,
    postpone_start=False,
    stop_event=[],
    requested_resources=requested_resources
)

single_run_b, activity_b, while_activity_b  = single_run_process(
    name="single_run_b",
    registry=registry,
    env=env,
    origin=origin,
    destination=destination,
    mover=NPRC_110_b,
    loader=crane_origin,
    unloader=crane_destination,
    postpone_start=False,
    stop_event=[],
    requested_resources=requested_resources
)


env.run()
registry

requesting <simpy.resources.resource.Resource object at 0x127be8f40> 0
requesting <simpy.resources.resource.Resource object at 0x127ccb040> 0
releasing <simpy.resources.resource.Resource object at 0x127be8f40> 1
releasing <simpy.resources.resource.Resource object at 0x127ccb040> 1
requesting <simpy.resources.resource.Resource object at 0x127be8f40> 0
requesting <simpy.resources.resource.Resource object at 0x127ccb040> 0
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 0 Crane Transferium Maasvlakte NPRC_110_a
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 0
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 1 Crane Transferium Maasvlakte NPRC_110_b
requesting <simpy.resources.resource.Resource object at 0x128008fa0> 0
releasing <simpy.resources.resource.Resource object at 0x127be8f40> 1
releasing <simpy.resources.resource.Resource object at 0x128008fa0> 1
releasing 1 Crane Transferium Maasvlakte for NPRC_110_a
releasing <simpy

releasing <simpy.resources.resource.Resource object at 0x127be8f40> 1
releasing <simpy.resources.resource.Resource object at 0x127ccb040> 1
requesting <simpy.resources.resource.Resource object at 0x127be8f40> 0
requesting <simpy.resources.resource.Resource object at 0x127ccb040> 0
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 0 Crane Transferium Maasvlakte NPRC_110_a
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 0
requesting <simpy.resources.resource.Resource object at 0x1266ed820> 1 Crane Transferium Maasvlakte NPRC_110_b
requesting <simpy.resources.resource.Resource object at 0x128008fa0> 0
releasing <simpy.resources.resource.Resource object at 0x127be8f40> 1
releasing <simpy.resources.resource.Resource object at 0x128008fa0> 1
releasing 1 Crane Transferium Maasvlakte for NPRC_110_a
releasing <simpy.resources.resource.Resource object at 0x1266ed820> 1
releasing <simpy.resources.resource.Resource object at 0x127ccb040> 1
not requested 0 Cr

{'name': {'single_run_a sailing empty': [<dtv_backend.activities.move_activity.MoveActivity at 0x127ccbee0>],
  'single_run_a loading': [<openclsim.model.shift_amount_activity.ShiftAmountActivity at 0x127ccb850>],
  'single_run_a sailing filled': [<dtv_backend.activities.move_activity.MoveActivity at 0x127ccbf10>],
  'single_run_a unloading': [<openclsim.model.shift_amount_activity.ShiftAmountActivity at 0x127ccbeb0>],
  'single_run_a sequence': [<openclsim.model.sequential_activity.SequentialActivity at 0x127be80a0>],
  'single_run_a': [<openclsim.model.while_activity.WhileActivity at 0x127be8040>],
  'single_run_b sailing empty': [<dtv_backend.activities.move_activity.MoveActivity at 0x127ccb670>],
  'single_run_b loading': [<openclsim.model.shift_amount_activity.ShiftAmountActivity at 0x127ccbfa0>],
  'single_run_b sailing filled': [<dtv_backend.activities.move_activity.MoveActivity at 0x127be89a0>],
  'single_run_b unloading': [<openclsim.model.shift_amount_activity.ShiftAmountActi

In [8]:
origin_node, distance = network_utilities.find_closest_node(env.FG, origin.geometry)
destination_node, distance = network_utilities.find_closest_node(env.FG, destination.geometry)

In [9]:


# Find route from A to B


result_objs = {
    'origin': origin, 
    'destination': destination, 
    'equipment': NPRC_110_a
}

logs = {}
for key, obj in result_objs.items():
    log_gdf = gpd.GeoDataFrame(pd.DataFrame(NPRC_110_a.log), geometry='Geometry')
    log_gdf['Timestamp'] = log_gdf['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')
    logs[key] = json.loads(log_gdf.to_json())

path = nx.dijkstra_path(env.FG, origin_node, destination_node, weight='Length')
path_gdf = dtv_backend.postprocessing.path2gdf(path, env.FG)

sites = []
for site in [origin, destination]:
    obj = {
        key: getattr(site, key) 
        for key in vars(site) 
        if key not in ["env", "resource", "container", "log", "wgs84"]
    }
    sites.append(obj)
sites_gdf = gpd.GeoDataFrame(sites, geometry='geometry')
    
result = {}
result["sites"] = json.loads(sites_gdf.to_json())
result["equipment"] = logs["equipment"]

path_features = geojson.loads(path_gdf.to_json())

path_geometry = shapely.ops.linemerge([
    shapely.geometry.asShape(feature['geometry'])
    for feature 
    in path_features['features']
])

result["path"] = shapely.geometry.mapping(path_geometry)
result["logs"] = logs


with open('sample-result.json', 'w') as f:
    json.dump(result, f)



In [14]:
def sort_path(path_gdf):
    """return sorted version of path_gdf"""
    path_gdf = path_gdf.copy()
    def invert(row):
        StartJunctionId = row.StartJunctionId
        EndJunctionId = row.EndJunctionId
        row.StartJunctionId = EndJunctionId
        row.EndJunctionId = StartJunctionId
        row.geometry = shapely.geometry.LineString(row.geometry.coords[::-1])
        return row
    # the start_node and StartJunctionId should match
    # if not, the path is inverted
    # we do this by id, because the node geometry and edge geometries do not match
    to_invert = path_gdf['start_node'] != path_gdf['StartJunctionId']
    path_gdf.loc[to_invert] = path_gdf.loc[to_invert].apply(invert, axis=1)
    return path_gdf

In [15]:
path_gdf = sort_path(path_gdf)
def first(line_string):
    return shapely.geometry.Point(line_string.coords[0])

def end(line_string):
    return shapely.geometry.Point(line_string.coords[-1])

# lookup  the start point
path_gdf['start_geom'] = path_gdf['StartJunctionId'].apply(
    lambda id: env.FG.nodes[id]['geometry']
)
# lookup the end point
path_gdf['end_geom'] = path_gdf['EndJunctionId'].apply(
    lambda id: env.FG.nodes[id]['geometry']
)

# this is very inaccurate
# TODO: improve snapping to align edges/node geometries
path_gdf['forward'] = path_gdf.apply(lambda row: first(row.geometry).almost_equals(row['start_geom'], decimal=2), axis=1)
# this list should now be empty
path_gdf.query('not forward')[['StartJunctionId', 'start_node', 'EndJunctionId', 'end_node', 'geometry', 'start_geom']]

,StartJunctionId,start_node,EndJunctionId,end_node,geometry,start_geom


In [16]:
import json
import geojson

with open('sample-result.json') as f:
    data = geojson.load(f)

In [17]:
with open('test.json', 'w') as f:
    f.write(json.dumps(data['path']))

## Visualize results of vessel planning    

In [18]:
import random

import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

def messageMapping(message):
    # note the spaces
    if ' loading ' in message.lower():
        simplified = 'loading'
    elif ' unloading ' in message.lower():
        simplified = 'unloading'
    elif 'sailing ' in message.lower():
        simplified = 'sailing'
    else:
        simplified = 'unknown activity'
    return simplified


def simplifyVesselLog(df):
    df['Message simplified'] = [messageMapping(m) for m in df['Message']]
    return df


def fixSailingActivityState(df):
    for ix in df.index:
        if 'sailing' in df.loc[ix, 'Message'].lower():
            if 'start' in df.loc[ix, 'Message'].lower():
                df.loc[ix, 'ActivityState'] = 'START'
            elif 'stop' in df.loc[ix, 'Message'].lower():
                df.loc[ix, 'ActivityState'] = 'STOP'
    return df


def modifyVesselLog(vessel):
    # extract the log to a dataframe
    df = pd.DataFrame(vessel.log)
    # fix the sailing ActivityState (make sure it reads START and STOP instead of UNKNOWN)
    df = fixSailingActivityState(df)
    
    # simplify the message (messages are too long, simplify to loading, unloading and sailing)
    df = simplifyVesselLog(df)
    
    # pass it back to the log
    new_log =  df.to_dict(orient='list')
    
    return new_log


def get_colors(n):
    """Get random colors for the activities."""
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append((r, g, b))
    return ret


def get_segments(df, activity, y_val):
    """Extract 'start' and 'stop' of activities from log."""
    x = []
    y = []
    for i in range(len(df)):
        if "START" in df["activity_state"][i] and df["log_string"][i] == activity:
            start = df.index[i]
        elif "STOP" in df["activity_state"][i] and df["log_string"][i] == activity:
            x.extend((start, start, df.index[i], df.index[i], df.index[i]))
            y.extend((y_val, y_val, y_val, y_val, None))
    return x, y


def vessel_planning(
    vessels, activities=None, colors=None, web=False, static=False, y_scale="text"
):
    """Create a plot of the planning of vessels."""
    # first simplify the vessel logs
    for vessel in vessels:
        vessel.log = modifyVesselLog(vessel)
        
    if activities is None:
        activities = []
        for obj in vessels:
            activities.extend(set(obj.log["Message simplified"]))
        # remove duplicates 
        activities = list(set(activities))

    if colors is None:
        C = get_colors(len(activities))
        colors = {}
        for i in range(len(activities)):
            colors[i] = f"rgb({C[i][0]},{C[i][1]},{C[i][2]})"

    # organise logdata into 'dataframes'
    dataframes = []
    names = []
    for vessel in vessels:
        if len(vessel.log["Timestamp"]) > 0:
            df = pd.DataFrame(
                {
                    "log_value": vessel.log["Value"],
                    "log_string": vessel.log["Message simplified"],
                    "activity_state": vessel.log["ActivityState"],
                },
                vessel.log["Timestamp"],
            )
            dataframes.append(df)
            names.append(vessel.name)

    df = dataframes[0]

    # prepare traces for each of the activities
    traces = []
    for i, activity in enumerate(activities):
        x_combined = []
        y_combined = []
        for k, df in enumerate(dataframes):
            y_val = -k if y_scale == "numbers" else names[k]
            x, y = get_segments(df, activity=activity, y_val=y_val)
            x_combined.extend(x)
            y_combined.extend(y)
        traces.append(
            go.Scatter(
                name=activity,
                x=x_combined,
                y=y_combined,
                mode="lines",
                hoverinfo="y+name",
                line=dict(color=colors[i], width=10),
                connectgaps=False,
            )
        )

    timestamps = []
    logs = [o.log["Timestamp"] for o in vessels]
    for log in logs:
        timestamps.extend(log)

    layout = go.Layout(
        title="Vessel planning",
        hovermode="closest",
        legend=dict(x=0, y=-0.2, orientation="h"),
        xaxis=dict(
            title="Time",
            titlefont=dict(family="Courier New, monospace", size=18, color="#7f7f7f"),
            range=[min(timestamps), max(timestamps)],
        ),
        yaxis=dict(
            title="Vessels",
            titlefont=dict(family="Courier New, monospace", size=18, color="#7f7f7f"),
        ),
    )

    if static is False:
        init_notebook_mode(connected=True)
        fig = go.Figure(data=traces, layout=layout)

        return iplot(fig, filename="news-source")
    else:
        return {"data": traces, "layout": layout}


In [19]:
vessel_planning([NPRC_110_a, NPRC_110_b])